In [1]:
import math
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModel

D:\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\pytorch\lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
D:\anaconda3\envs\pytorch\lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
file_path = "语义增强文本.xlsx"
data = pd.read_excel(file_path)

In [3]:
# 以 PubMedBERT 为例
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")
model = AutoModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")

In [4]:
texts = data['输出结果']

In [5]:
lengths = []
for text in texts:
    enc = tokenizer(text, return_tensors='pt', padding=False, truncation=False)
    lengths.append(enc['input_ids'].size(1)) 

In [6]:
avg_len = math.ceil(sum(lengths) / len(lengths))
print(f"计算得到的平均 token 数（max_length）：{avg_len}")

计算得到的平均 token 数（max_length）：189


In [7]:
def get_text_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=189)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :]

In [8]:
text_embeddings = np.array([get_text_embedding(text) for text in data['输出结果']])

In [9]:
np.save('PubMedBERT.npy', text_embeddings.squeeze())

In [24]:
text_embeddings = np.load('PubMedBERT.npy')